In [35]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
import ipywidgets as wg

# Ising model function definitions

In [36]:
def energy_calc(lattice):
    ''' Calculated total energy of the lattice.
    
        Parameters:
            lattice: [N,N] numpy array containing only 1 and -1 values representing the spin configuration.
        
        Returns: dtype float
            Energy of the spin configuration'''
    
    a = lattice[:,1:]*lattice[:,:N-1]
    b = lattice[:N-1,:]*lattice[1:,:]
    # To ensure periodic boundary conditions the last column is multiplied with the first column of the lattice array. 
    # Same is done with the rows.
    c = lattice[0]*lattice[N-1]
    d = lattice[:,0]*lattice[:,N-1]
    # Energy:
    E = -1*(np.sum(a)+np.sum(b)+np.sum(c)+np.sum(d))
    return(E)

In [37]:
def energy_nn(lattice,x,y):
    ''' Calculate nearest neighbour interaction energy of 1 spin in the lattice.
    
        Parameters:
            lattice: dtype numpy array
                [N,N] numpy array containing only 1 and -1 values representing the spin configuration.
            
            x, y: dtype int
                The coordinates of the of the spin of interest
        
        Returns: dtype float
             nearest neighbour interaction energy of the selected spin '''
    
    s = lattice[x,y]
    nn = lattice[(x+1)%N,y] + lattice[(x-1)%N,y] + lattice[x,(y+1)%N] + lattice[x,(y-1)%N]
    E = -1*nn*s
    return(E)

In [38]:
def specific_heat(x):
    ''' Calculates specific heat.
        
        Parameters: 
            x: dtype numpy array
                One dimensional array containing the energy data (or equivalent quantity)
            
        Returns: dtype numpy array
            One dimensional array containing the specific heat data'''
    
    cv = (np.var(x, axis=0))/(T**2)
    return(cv)

In [39]:
def susceptiblity(mag):
    ''' Calculates specific heat.
        
        Parameters: 
            mag: dtype numpy array
                One dimensional array containing the magnetization data
            
        Returns: dtype numpy array
            One dimensional array containing the susceptibility data'''
    
    susc = (np.var(mag))/T
    return(susc)

In [40]:
def metropolis(lattice,E,M):
    ''' Implement metropolis algorithm for a single spin flip.
    
        Parameters: 
            lattice: dtype numpy array
                [N,N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    # Select random spin (propose trial lattice with probability 1/L^2):
    x = np.random.randint(0,N)
    y = np.random.randint(0,N)
    trial_lattice = np.copy(lattice)
    trial_lattice[x,y] *= -1
    # Calculate energy difference between the old and trial configurations:
    dE = energy_nn(trial_lattice,x,y) - energy_nn(lattice,x,y)
    # Implement second stage of the metropolis algorithm:
    if dE <= 0:
        E = E + dE
        M = M - lattice[x,y] + trial_lattice[x,y] # substract old spin value and add new spin value to the magnetisation
        lattice[x,y] = trial_lattice[x,y]
    else:
        r = np.random.uniform(0,1)
        #p=np.exp(-dE/tau)
        if dE == 4:
            p = exp[1]
        else:
            p = exp[0]
        if r < p:
            E = E + dE
            M = M - lattice[x,y] + trial_lattice[x,y] # substract old spin value and add new spin value to the magnetisation
            lattice[x,y] = trial_lattice[x,y]
    return(lattice,E,M)

In [41]:
def Wolffbacktrack(x,y,cluster,visited):
    ''' Part of the Wolff algorithm, it checks whether a spin is already in the cluster'''
    if cluster[x,y]:
        return False
    if visited[x,y]:
        return True#False
    return True

def grow_cluster(lattice,x,y,ClusterSpin,E,M, cluster,visited):
    """ Starting from a randomly chosen spin, creates a cluster with the Wolff Algorithm.
        
        Parameters:
            lattice: dtype numpy array
                The [N,N] array representing the spin configuration
            
            x, and y: dtype int
                The coordinates of the spin that is added to the cluster
                
            ClusterSpin: dtype int
                The spin value of the cluster
            
            E and M: dtype int or float
                The energy and magnetization of the lattice
            
            cluster: dtype boolean numpy array
                [N,N] boolean numpy array indicating which spins are in the cluster
                
            visited: dtype boolean numpy array
                [N,N] boolean numpy array indicating which spins have already been trialed by the algorithm
                
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization"""
    
    old_E = energy_nn(lattice,x,y)
    lattice[x,y] *=-1
    dE = energy_nn(lattice,x,y) - old_E
    E = E + dE
    M = M - lattice[x,y]*-1 +lattice[x,y]
    #cluster.append([x,y])
    cluster[x,y] = True
    if Wolffbacktrack((x+1)%N,y, cluster, visited):
        E, M = try_add(lattice,(x+1)%N,y,ClusterSpin,E,M, cluster,visited)
    if Wolffbacktrack((x-1)%N,y, cluster,visited):
        E, M = try_add(lattice,(x-1)%N,y,ClusterSpin,E,M, cluster,visited)
    if Wolffbacktrack(x,(y+1)%N, cluster,visited):
        E, M = try_add(lattice,x,(y+1)%N,ClusterSpin,E,M, cluster,visited)
    if Wolffbacktrack(x,(y-1)%N, cluster,visited):
        E, M = try_add(lattice,x,(y-1)%N,ClusterSpin,E,M, cluster,visited)
    return(lattice,E,M)

def try_add(lattice,x,y,ClusterSpin,E,M, cluster, visited):
    """Checks of neighbor belongs to cluster, if so then add neighbor to cluster with a certain probability"""
    
    s_help = lattice[x,y]
    #global tau
    if ((s_help<0) != (ClusterSpin<0)):
        r = np.random.uniform(0,1)
        if r < (1-A_wolff):
            lattice, E, M = grow_cluster(lattice,x,y,ClusterSpin,E,M, cluster,visited)
        else:
            visited[x,y] = True
    return(E,M)

def Wolff(lattice,E,M):
    ''' Implements the wolff algorithm for a single timestep.
        
        Parameters: 
            lattice: dtype numpy array
                [N,N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    x = np.random.randint(0,N)
    y = np.random.randint(0,N)
    cluster = np.full((N,N), False, dtype=bool)
    visited = np.full((N,N), False, dtype=bool)
    clusterspin = lattice[x,y]*-1
    lattice, E, M = grow_cluster(lattice,x,y,clusterspin, E, M, cluster,visited)
    return(lattice,E,M)

# Correspondence function definitions

In [42]:
def network_energy_calc(N_edges, N_l):
    # N_l is the total number of possible cross-edges (possible nn interactions)
    nwEl = (2*N_edges)-(N_l)
    return(nwEl)

In [43]:
def number_of_RGsteps(n):
    tel = 0
    neG_baseline = n**2-1
    lat_size = n
    while n > 1:
        if n%2 != 0:
            raise ValueError("%d is not a power of the blocking size 2"%(lat_size))
        tel += 1
        n = n/2
        neG_baseline += n**2
    return (tel, neG_baseline)

In [44]:
def RGlats_skeleton():
    """ Creates the 'bare' shells of the RGb graph. That is, a list of spin lattices of decreasing size in accordance with the 
        branching factor 2, that serve as 'skeleton lattices' that can be modified by the RGb procedure
        
        Returns: 
            lats: dtype list
                list of lattice arrays. They serve as the initial spin lattices that constitute the shells of the RGb graph
    """
    lats = []; lats.append(np.zeros((N,N)))
    n = N # length of shell
    while n > 1:
        n //= 2
        new_lat = np.zeros((n,n))
        lats.append(new_lat)
    return(lats)

In [45]:
def boundary_pos():
    """ Part of RGgraph_skeleton. Creates dictionary, which labels the spins of lattice (assigning a number from 0 to N^2) and 
        registers their position in the lattice
        
        Returns:
            pos: dtype dictionary
                dictionary of (boundary) spins which registers their position in the lattice
    """
    tel = 0
    pos = {i: [] for i in range(N**2)}
    for j in range(N):
        for i in range(N):
            pos[tel] = [i,j,0]
            tel += 1
    return(pos)

In [46]:
def RGgraph_skeleton():
    """ Creates boundary graph (collection of nodes) corresponding to the spin lattice, and the 'bare' tree graph
        
        Returns:
            bulkG: dtype networkx multigraph
                'bare' tree graph, i.e. the boundary + bulk tree graph that serves as a skeleton for the RGb graph
                
            pos: dtype dictionary
                extended boundary position dictionary to a dictionary of all the nodes/spins of the network, i.e. all 
                nodes/spins of the network are labeled by a number and the dictionary registers their (3D) position in space
            
            lats_idx: dtype list
                list of arrays that contain the node/spin labels fo each shell lattice similar to the position dictionary. 
    """
    shell = 0; dz =1
    n = N; new_n = n
    pos = boundary_pos()
    lats_idx = []
    
    # Boundary graph
    bdG = nx.MultiGraph()
    for i in range(N**2):
        bdG.add_node(i)
    # Bulk graph
    bulkG = bdG.copy()
    lat_idx = np.arange(N**2).reshape(N,N)#np.transpose(np.arange(N**2).reshape(N,N))
    lats_idx.append(lat_idx)
    #print(lat_idx)
    
    while n > 1:
        new_n //= 2
        new_lat_idx = np.zeros((new_n,new_n))
        for i in np.arange(0,n,2):
            for j in np.arange(0,n,2):
                n_Gr = nx.number_of_nodes(bulkG); bulkG.add_node(n_Gr)
                block = lat_idx[i:i+2,j:j+2].reshape(4)
                new_pos = []
                for k in block:
                    bulkG.add_edge(n_Gr,k)
                    new_pos.append(pos[k])
                new_lat_idx[i//2,j//2] = n_Gr
                new_pos = list(np.round(np.mean(np.array(new_pos),axis=0),1))
                new_pos[2] = (shell+1)*dz
                pos.update({n_Gr: new_pos})
        lat_idx = np.transpose(new_lat_idx)
        lats_idx.append(lat_idx)
        n = new_n
        shell += 1
        #print(lat_idx)
        
    return(bulkG, pos, lats_idx)

In [47]:
def create_frozen_map(lat):
    """ Creates a boolean map of (freezes) all bonds (nn interactions) in the lattice
    
        Input:
            lat: dtype numpy array
                [N,N] array containing only 1 and -1 values representing the spin configuration
            
        Returns:
            f_map_x: dtype numpy array
                [N,N] boolean array containing True if nn bond is present in the x-direction (horizontal), or False if not.
                
            f_map_y: dtype numpy array
                [N,N] boolean array containing True if nn bond is present in the y-direction (vertical), or False if not.
    """
    lat = np.array(lat)
    a = lat[:,1:]-lat[:,:-1]; b = lat[:,0]-lat[:,-1]
    b = np.reshape(b,(len(lat),1))
    c = np.append(a,b,axis=1)
    f_map_y = (c == 0)
    # same for y
    a = lat[1:]-lat[:-1]; b = lat[0]-lat[-1]
    b = np.reshape(b,(1,len(lat)))
    c = np.append(a,b,axis=0)
    f_map_x = (c == 0)
    return(f_map_x, f_map_y)

In [48]:
def construct_graph(lat, lats, lats_idx, Gr):
    """ Construct graph from spin lattice according to the RGb procedure
        
        Input: 
            lat: dtype numpy array
                [N,N] array containing only 1 and -1 values representing the spin configuration
            
            lats: dtype list
                list containing the initial spin lattices that constitutes the shells of the graph (i.e. as constructed by the 
                function 'RGlats_skeleton')
                
            lats_idx: dtype list
                list of arrays that contain the node/spin labels of the spin lattices (shells)
            
            Gr: dtype networkx multigraph
                'bare' tree graph that serves as the 'skeleton' for the RGb graph (i.e. as constructed by the function 
                'RGgraph_skeleton')
            
        Returns: 
            Gr: dtype networkx multigraph
                RGb graph
    """
    n = N; shell = 0
    Gr = Gr.copy()
    while shell < len(lats):
        lats[shell] = lat
        if shell == (len(lats)-1):
            break
        # Update RG graph accordingly:
        frmx, frmy = create_frozen_map(lat)
        idx = np.where(frmx==False)
        for i,j in zip(idx[0], idx[1]):
            k = lats_idx[shell][i,j]
            l = lats_idx[shell][(i+1)%n,j]
            Gr.add_edge(k,l)
        idx = np.where(frmy==False)
        for i,j in zip(idx[0], idx[1]):
            k = lats_idx[shell][i,j]
            l = lats_idx[shell][i,(j+1)%n]
            Gr.add_edge(k,l) 
        # Do RG step:
        old_lat = lat
        n //= 2
        lat = lat.reshape(n,2,n,2) 
        lat = lat.sum(axis=(1,3))
        idx_zeros = np.where(lat==0)
        old_idx_zeros = tuple([2*i for i in idx_zeros])
        lat[idx_zeros] = old_lat[old_idx_zeros] #np.random.choice([1,-1], size=len(idx_zeros[1]))
        lat = lat/abs(lat)
        
        shell += 1
    return(Gr)

In [49]:
def bulk_plot(ax, Gr, pos):
    """ Creates 3D plot of the network (boundary+bulk), i.e. its nodes and edges
    
        Input:
            ax: dtype matplotlib axis (projection 3d)
            
            Gr: dtype networkx graph
                the RGb graph
            
            pos: dtype dictionary
                position dictionary of the nodes/spins of the network    
    """
    hs = 1; vs = 1.25
    ax.cla()
    # Loop on the pos dictionary to extract the x,y,z coordinates of each node
    for key, value in pos.items():
        xi = value[0]; yi = value[1]; zi = value[2]
        ax.scatter(hs*xi, hs*yi, vs*zi, s=10, c='r')
        ax.text(hs*xi,hs*yi,vs*zi,key)
        ax.set_zlim3d(0,4)
    
    # Loop on the list of edges to get the x,y,z, coordinates of the connected nodes
    # Those two points are the extrema of the line to be plotted
    for i,j in enumerate(Gr.edges()):
        x = np.array((pos[j[0]][0], pos[j[1]][0]))
        y = np.array((pos[j[0]][1], pos[j[1]][1]))
        z = np.array((pos[j[0]][2], pos[j[1]][2]))
    
        # Plot the connecting lines
        ax.plot(hs*x, hs*y, vs*z, c='k',linewidth=0.8,alpha=0.7)
        
    ax.set_axis_off()
    return()

# Exhibition

In [50]:
def exhibition(temp, save):
    t_sim = 15
    global T
    T = temp
    #global exp
    #exp = np.array([np.exp(-8/T),np.exp(-4/T),1,np.exp(4/T),np.exp(8/T)]) # Store possible values of the acceptance probability (only
    # need first two values for metropolis)
    global A_wolff
    A_wolff = np.exp(-2/T) # Needed for acceptance value for Wolff
    
    # Check if N is a power of the block size
    nRGsteps, neGbase = number_of_RGsteps(N)
    
    # Initialize lattice
    lattice = -1*np.ones((N,N))  
    #lattice = np.random.choice([1,-1],size=(N,N))
    
    # Initialize RG lattices
    lattices = RGlats_skeleton()

    # Initialize bulk graph
    bulkG, positions, lattices_idx = RGgraph_skeleton()

    # Calculate energy and magnetisation by performing a sweep over the whole lattice (only once, as at each step it is updated)
    E = energy_calc(lattice); M = np.sum(lattice) 

    # Initialise figure
    %matplotlib notebook
    fig = plt.figure(figsize=(10,5)); plt.title('N=%d, T=%.2f'%(N**2,temp)); plt.axis('off') 
    grid = plt.GridSpec(10, 10, hspace=0.2, wspace=0.2)
    ax1 = fig.add_subplot(grid[2:8,:3]); ax1.set_axis_off()
    ax2 = fig.add_subplot(grid[:,3:],projection='3d'); ax2.set_zlim3d(0,4); ax2.view_init(azim=-63,elev=17)

    # Create boundary plot
    x, y = np.meshgrid(range(N),range(N))#, indexing='ij')
    # Add labels
    for i in range(N**2):
        ax1.annotate(i,(positions[i][0],positions[i][1]))

    for j in range(t_sim):
        # Perform one MCS using metropolis algorithm:
        #for k in range(N**2):
            #lattice, E, M = metropolis(lattice,E,M) 
            
        # Perform cluster flip using wolff algorithm:
        lattice, E, M = Wolff(lattice,E,M) 
        
        # Update boundary
        ax1.scatter(x,y,s=100,c=lattice)
    
        # Update bulk
        graph = construct_graph(lattice, lattices, lattices_idx, bulkG.copy())
        bulk_plot(ax2, graph, positions)
    
        fig.canvas.draw()
        #time.sleep(0.1)
    #print(graph.edges())
    if save:
        plt.savefig('RG2DcorrespondenceN=%dT=%d'%(N**2,temp))

In [51]:
N = 4
wg.interact_manual(exhibition, temp=(0.1,5,0.01), save=False)

interactive(children=(FloatSlider(value=2.5500000000000003, description='temp', max=5.0, min=0.1, step=0.01), …

<function __main__.exhibition(temp, save)>